In [1]:
import copy
import math
import random
from abc import ABC, abstractmethod
from string import ascii_lowercase, digits

from gen_transform import gen_transform
from loop_transform import loop_transform
from loop_transform_copy import loop_transform_copy

# flake8 swears that imported modules aren't used
gen_transform
loop_transform
loop_transform_copy

%load_ext memory_profiler

## 1. Чтение/запись в словарь.

In [2]:
class AbstractDict(ABC):
    @abstractmethod
    def get(self):
        ...

In [3]:
class Benchmark_1:
    def __init__(self, n_keys: int = 2**13):
        '''
        Creates sets of `n_keys` keys for futher benchmark runs.

        There are keys from the dictionary and made up keys.
        To make it more real existent keys appear 4 times more
        often
        '''
        random.seed(42)
        n_non_exst_keys = n_keys // 2**6
        n_keys_from_dict = n_keys // 2**4

        '''keys for storing in dict
        `tuple` is used to keep keys between runs'''
        self.keys = tuple(self._key_gen(n_keys))

        '''keys to choose from for benchmarking `get` method
        it includes some random keys from dict's actual keys and made up
        ones ('__nnn__')'''
        keys_to_check = tuple((random.choices(self.keys, k=n_keys_from_dict)
                               + ['__nnn__']*n_non_exst_keys))

        n_ops = n_keys * 2**4  # number of keys to get during benchmark
        self.keys_to_check = tuple(random.choice(
            keys_to_check) for i in range(n_ops))

    def _key_gen(self, n_keys: int):
        '''
        Generates random keys from alphanumeric characters
        with length from 0 to 10.
        '''
        keys_alphabet = ascii_lowercase + digits
        for i in range(n_keys):
            key_len = random.randint(0, 10)
            key = ''.join(random.choices(keys_alphabet, k=key_len))
            yield key

    def __call__(self, D: AbstractDict):
        d = D()
        for key in self.keys:
            d[key] = key

        for key in self.keys_to_check:
            d.get(key)

In [4]:
benchmark_1 = Benchmark_1()

### 1) Встроенный `dict`, использование `try/except`

In [5]:
class Dict_1(dict, AbstractDict):
    def get(self, key):
        try:
            return self[key]
        except KeyError:
            return None

In [6]:
%timeit benchmark_1(Dict_1)

38.7 ms ± 4.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%prun benchmark_1(Dict_1)

         131076 function calls in 0.163 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   131072    0.086    0.000    0.086    0.000 50095481.py:2(get)
        1    0.077    0.077    0.163    0.163 1638636319.py:26(__call__)
        1    0.000    0.000    0.163    0.163 <string>:1(<module>)
        1    0.000    0.000    0.163    0.163 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [8]:
%memit benchmark_1(Dict_1)

peak memory: 76.54 MiB, increment: 0.12 MiB


### 2) Встроенный `dict`, использование `if`

In [9]:
class Dict_2(dict, AbstractDict):
    def get(self, key):
        if key in self:
            return self[key]
        return None

*Если использовать условие проверки `key in self.keys()`, то время работы значительно увеличивается, т. к. на каждой итерации выполняется вызов функции `keys`

In [10]:
%timeit benchmark_1(Dict_2)

34.9 ms ± 3.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%prun benchmark_1(Dict_2)

         131076 function calls in 0.135 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.067    0.067    0.134    0.134 1638636319.py:26(__call__)
   131072    0.067    0.000    0.067    0.000 2182802591.py:2(get)
        1    0.000    0.000    0.135    0.135 <string>:1(<module>)
        1    0.000    0.000    0.135    0.135 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

### 3) Встроенный `dict`, использование метода `get`

In [12]:
class Dict_3(dict, AbstractDict):
    ...

In [13]:
%timeit benchmark_1(Dict_3)

14.3 ms ± 844 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%prun benchmark_1(Dict_3)

         131076 function calls in 0.132 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.083    0.083    0.132    0.132 1638636319.py:26(__call__)
   131072    0.049    0.000    0.049    0.000 {method 'get' of 'dict' objects}
        1    0.000    0.000    0.132    0.132 {built-in method builtins.exec}
        1    0.000    0.000    0.132    0.132 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [15]:
%memit benchmark_1(Dict_3)

peak memory: 76.58 MiB, increment: 0.04 MiB


**Выводы:**

* Как и ожидалось, лучше использовать встроенный метод словаря, он значительно быстрее других решений.
* Первый вариант с обработкой исключения работает так же или немного медленнее, чем вариант с условием
* Не нужно явно вызывать метод `keys` у словаря при проверке наличия элемента среди его ключей



## 2. К каждому элементу списка применить какое-либо преобразование.

In [16]:
class Benchmark_2:
    def __init__(self, n_elems=2**17):
        random.seed(42)
        self.elems = list(random.uniform(-2**20, 2**20)
                          for i in range(n_elems))

    @staticmethod
    def func(x: float):
        '''Sample transformation function'''
        return math.log2(math.fabs(x))

    def __call__(self, transform):
        '''It is supposed that original list won't be changed'''
        transformed = transform(Benchmark_2.func, self.elems)
        return transformed

In [17]:
benchmark_2 = Benchmark_2()

### 1.1) Цикл `for` с `append`

In [18]:
%%writefile loop_transform.py
def loop_transform(func, lst):
    res = []
    for item in lst:
        res.append(func(item))
    
    return res

Overwriting loop_transform.py


In [19]:
%timeit benchmark_2(loop_transform)

80.3 ms ± 5.92 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%prun benchmark_2(loop_transform)

         524293 function calls in 0.310 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   131072    0.099    0.000    0.195    0.000 407990253.py:7(func)
        1    0.090    0.090    0.309    0.309 loop_transform.py:1(loop_transform)
   131072    0.054    0.000    0.054    0.000 {built-in method math.log2}
   131072    0.042    0.000    0.042    0.000 {built-in method math.fabs}
   131072    0.023    0.000    0.023    0.000 {method 'append' of 'list' objects}
        1    0.001    0.001    0.310    0.310 <string>:1(<module>)
        1    0.000    0.000    0.310    0.310 {built-in method builtins.exec}
        1    0.000    0.000    0.309    0.309 407990253.py:12(__call__)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [21]:
%mprun -f loop_transform benchmark_2(loop_transform)

Filename: c:\Users\User\Documents\epam\python_mentoring\mikhail_shchukin_python_hw\hw3_profiling\loop_transform.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     1     83.9 MiB     83.9 MiB           1   def loop_transform(func, lst):
     2     83.9 MiB      0.0 MiB           1       res = []
     3     85.3 MiB      0.4 MiB      131073       for item in lst:
     4     85.3 MiB      1.1 MiB      131072           res.append(func(item))
     5                                             
     6     85.3 MiB      0.0 MiB           1       return res

### 1.2) Копирование с циклом `for`

In [22]:
%%writefile loop_transform_copy.py
def loop_transform_copy(func, lst):
    res = lst.copy()
    for i, item in enumerate(lst):
        res[i] = func(item)
    
    return res

Overwriting loop_transform_copy.py


In [23]:
%timeit benchmark_2(loop_transform_copy)

82.1 ms ± 8.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%prun benchmark_2(loop_transform_copy)

         393222 function calls in 0.247 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   131072    0.093    0.000    0.183    0.000 407990253.py:7(func)
        1    0.061    0.061    0.246    0.246 loop_transform_copy.py:1(loop_transform_copy)
   131072    0.050    0.000    0.050    0.000 {built-in method math.log2}
   131072    0.040    0.000    0.040    0.000 {built-in method math.fabs}
        1    0.001    0.001    0.001    0.001 {method 'copy' of 'list' objects}
        1    0.001    0.001    0.247    0.247 <string>:1(<module>)
        1    0.000    0.000    0.247    0.247 {built-in method builtins.exec}
        1    0.000    0.000    0.246    0.246 407990253.py:12(__call__)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [25]:
%mprun -f loop_transform_copy benchmark_2(loop_transform_copy)

Filename: c:\Users\User\Documents\epam\python_mentoring\mikhail_shchukin_python_hw\hw3_profiling\loop_transform_copy.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     1     84.9 MiB     84.9 MiB           1   def loop_transform_copy(func, lst):
     2     86.0 MiB      1.0 MiB           1       res = lst.copy()
     3     86.0 MiB  -7459.7 MiB      131073       for i, item in enumerate(lst):
     4     86.0 MiB  -7459.6 MiB      131072           res[i] = func(item)
     5                                             
     6     85.9 MiB     -0.1 MiB           1       return res

### 2) Функция `map`

In [26]:
%timeit benchmark_2(map)

434 ns ± 55 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [27]:
%prun benchmark_2(map)

         4 function calls in 0.000 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 407990253.py:12(__call__)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

### 3) Генератор

In [28]:
%%writefile gen_transform.py
def gen_transform(func, it):
    for item in it:
        yield func(item)

Overwriting gen_transform.py


In [29]:
%timeit benchmark_2(gen_transform)

550 ns ± 55.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [30]:
%prun benchmark_2(loop_transform)

         524293 function calls in 0.304 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   131072    0.098    0.000    0.191    0.000 407990253.py:7(func)
        1    0.088    0.088    0.303    0.303 loop_transform.py:1(loop_transform)
   131072    0.053    0.000    0.053    0.000 {built-in method math.log2}
   131072    0.040    0.000    0.040    0.000 {built-in method math.fabs}
   131072    0.025    0.000    0.025    0.000 {method 'append' of 'list' objects}
        1    0.002    0.002    0.304    0.304 <string>:1(<module>)
        1    0.000    0.000    0.304    0.304 {built-in method builtins.exec}
        1    0.000    0.000    0.303    0.303 407990253.py:12(__call__)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [31]:
%mprun -f gen_transform benchmark_2(gen_transform)

Filename: c:\Users\User\Documents\epam\python_mentoring\mikhail_shchukin_python_hw\hw3_profiling\gen_transform.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     1     84.9 MiB     84.9 MiB           1   def gen_transform(func, it):
     2                                             for item in it:
     3                                                 yield func(item)

**Выводы:**
* Стоит учитывать то, нужно ли изменить сам список или создать копию и каким образом должны быть доступны элементы после модификации
* В целом, если нужно вернуть копию списка в виде списка, то лучше, похоже, скопировать его, и изменить элементы в копии, нежели создавать новый элемент и добавлять его в новый список, т.к. копирование выполняется один раз (лучше использовать метод `copy`, а не `[:]`), а `append` будет вызван для каждого элемента, и если элементов много, будет несколько вызовов для увеличения вместимости списка, хотя конечный размер нам известен
* Если же приемлем последовательный доступ к элементам модифицированной последовательности, то лучше использовать генератор или функцию `map`, т.к. они ленивые. В данном бенчмарке не удалось проверить, какой итератор быстрее отдает значения, но `map` создала итератор быстрее

## 3. Отсортировать список.

In [32]:
class Benchmark_3:
    def __init__(self, n_elems=2**17):
        random.seed(42)
        self.sequenc_to_sort = list(range(n_elems))
        random.shuffle(self.sequenc_to_sort)

    def __call__(self, sort):
        sorted_list = sort(self.sequenc_to_sort)
        return sorted_list

In [33]:
benchmark_3 = Benchmark_3()

### 1) Встроенная функция `sorted`

In [56]:
%timeit benchmark_3(sorted)

42.9 ms ± 2.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [57]:
%prun benchmark_3(sorted)

         5 function calls in 0.077 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.076    0.076    0.076    0.076 {built-in method builtins.sorted}
        1    0.001    0.001    0.077    0.077 <string>:1(<module>)
        1    0.000    0.000    0.077    0.077 {built-in method builtins.exec}
        1    0.000    0.000    0.076    0.076 199360473.py:7(__call__)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [58]:
%memit benchmark_3(sorted)

peak memory: 102.38 MiB, increment: 1.00 MiB


### 2) Метод списка `sort`

In [59]:
def list_sort(lst):
    res = list(lst)
    res.sort()
    return res

In [60]:
%timeit benchmark_3(list_sort)

40.9 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [61]:
%prun benchmark_3(list_sort)

         6 function calls in 0.076 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.066    0.066    0.066    0.066 {method 'sort' of 'list' objects}
        1    0.007    0.007    0.074    0.074 1067735932.py:1(list_sort)
        1    0.002    0.002    0.076    0.076 <string>:1(<module>)
        1    0.000    0.000    0.076    0.076 {built-in method builtins.exec}
        1    0.000    0.000    0.074    0.074 199360473.py:7(__call__)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [62]:
%memit benchmark_3(list_sort)

peak memory: 101.39 MiB, increment: 1.00 MiB


**Выводы:**
* Метод `sort` работает быстрее функции `sorted`, его стоит использовать, особенно, если нужно отсортировать список inplace.

## 4. Распаковать вложенный список.

In [41]:
def identity(x):
    return x


def pack(x):
    return [x]


def where_to_append(el, prev_l):
    '''
    Determines to which list to append.

    :param el: Last element of outer list
    :param prev_l: Outer list
    :return: List to which element will be appended
    '''
    if not isinstance(el, list):
        return prev_l  # prev_l is always a list
    else:
        return random.choice((where_to_append(el[-1], el), el))

In [42]:
pack_or_identity = (identity, pack, pack)

ri = iter(range(2**4))
test_list = [next(ri)]
for i in ri:
    wtp = where_to_append(test_list, None)
    wtp.append(random.choice(pack_or_identity)(i))

test_list

[0, [1, [2], [3, [4, 5]]], [6, [7]], [8], [9], 10, [11, [12, [13]], [14]], 15]

In [43]:
class Benchmark_4:
    def __init__(self, n_elems=2**17):
        random.seed(42)
        pack_or_identity = (identity, pack, pack)  # 2 packs per 1 identity

        ri = iter(range(n_elems))
        # we need at least one element in initial list so `ll[-1]` works
        self.ll = [next(ri)]
        for item in ri:
            wtp = where_to_append(self.ll, None)  # ll is a list
            wtp.append(random.choice(pack_or_identity)(item))

    def __call__(self, flatten):
        # some funcs change given list inplace
        return flatten(copy.deepcopy(self.ll))

In [44]:
benchmark_4 = Benchmark_4()

### 1) Цикл `for` и слайсы

In [45]:
def flatten(ll: list):
    for i, item in enumerate(ll):
        if isinstance(item, list):
            ll[i:i+1] = flatten(item)
    return ll

In [46]:
flatten(copy.deepcopy(test_list))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [47]:
%timeit benchmark_4(flatten)

1.1 s ± 156 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
%prun benchmark_4(flatten)

         2027494 function calls (1634621 primitive calls) in 2.215 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  87268/1    0.821    0.000    0.877    0.877 4083973429.py:1(flatten)
 218340/1    0.668    0.000    1.330    1.330 copy.py:128(deepcopy)
  87268/1    0.272    0.000    1.330    1.330 copy.py:201(_deepcopy_list)
   436680    0.124    0.000    0.124    0.000 {method 'get' of 'dict' objects}
    87268    0.088    0.000    0.125    0.000 copy.py:243(_keep_alive)
   392877    0.084    0.000    0.084    0.000 {built-in method builtins.id}
   305606    0.062    0.000    0.062    0.000 {method 'append' of 'list' objects}
   281111    0.056    0.000    0.056    0.000 {built-in method builtins.isinstance}
   131072    0.030    0.000    0.030    0.000 copy.py:182(_deepcopy_atomic)
        1    0.007    0.007    2.214    2.214 3045739282.py:13(__call__)
        1    0.001    0.001    2.215    2.215 <string>:1(<module>)
 

In [49]:
%memit benchmark_4(flatten)

peak memory: 109.76 MiB, increment: 9.63 MiB


### 2) Генератор

In [50]:
def flatten_gen(ll: list):
    for item in ll:
        if isinstance(item, list):
            yield from flatten_gen(item)
        else:
            yield item


def flatten2(ll: list):
    return list(flatten_gen(ll))

In [51]:
flatten2(test_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [52]:
test_list

[0, [1, [2], [3, [4, 5]]], [6, [7]], [8], [9], 10, [11, [12, [13]], [14]], 15]

In [53]:
%timeit benchmark_4(flatten2)

630 ms ± 66.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
%prun benchmark_4(flatten2)

         2245834 function calls (1702922 primitive calls) in 2.215 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
 218340/1    0.981    0.000    1.947    1.947 copy.py:128(deepcopy)
  87268/1    0.397    0.000    1.947    1.947 copy.py:201(_deepcopy_list)
368379/131073    0.193    0.000    0.228    0.000 4247115795.py:1(flatten_gen)
   436680    0.182    0.000    0.182    0.000 {method 'get' of 'dict' objects}
    87268    0.129    0.000    0.183    0.000 copy.py:243(_keep_alive)
   392877    0.125    0.000    0.125    0.000 {built-in method builtins.id}
   305606    0.089    0.000    0.089    0.000 {method 'append' of 'list' objects}
   131072    0.044    0.000    0.044    0.000 copy.py:182(_deepcopy_atomic)
   218339    0.035    0.000    0.035    0.000 {built-in method builtins.isinstance}
        1    0.029    0.029    0.258    0.258 4247115795.py:9(flatten2)
        1    0.009    0.009    2.214    2.214 3045739282.py:1

In [55]:
%memit benchmark_4(flatten2)

peak memory: 108.25 MiB, increment: 6.00 MiB


**Выводы:**
* Лучше использовать решение с генератором, т.к. оно более эффективно как по времени, так и по памяти, также его можно использовать без предварительного копирования списка